In [4]:
import ee
import geemap

In [5]:
# Create a default map
Map = geemap.Map()

# Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B5'])
ndwiViz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}
Map.setCenter(-122.1899, 37.5010, 10)  # San Francisco Bay
Map.addLayer(ndwi, ndwiViz, 'NDWI', False)

# Display the map
Map



Map(center=[37.501, -122.1899], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

## Masking

You can use image.updateMask() to set the opacity of individual pixels based on where pixels in a mask image are non-zero. Pixels equal to zero in the mask are excluded from computations and the opacity is set to 0 for display. The following example uses an NDWI threshold (see the Relational Operations section for information on thresholds) to update the mask on the NDWI layer created previously:

In [6]:
# Mask the non-watery parts of the image, where NDWI < 0.4.
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
Map.addLayer(ndwiMasked, ndwiViz, 'NDWI masked')
Map

Map(bottom=101876.0, center=[37.501, -122.1899], controls=(WidgetControl(options=['position', 'transparent_bg'…

## Visualization images

Use the image.visualize() method to convert an image into an 8-bit RGB image for display or export. For example, to convert the false-color composite and NDWI to 3-band display images, use:

In [7]:
# Create a default map
Map = geemap.Map()

# Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Create an NDWI image, define visualization parameters and display.
ndwi = image.normalizedDifference(['B3', 'B5'])
ndwiViz = {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}
Map.setCenter(-122.1899, 37.5010, 10)  # San Francisco Bay
Map.addLayer(ndwi, ndwiViz, 'NDWI', False)

# Mask the non-watery parts of the image, where NDWI < 0.4.
ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
Map.addLayer(ndwiMasked, ndwiViz, 'NDWI masked', False)

# Create visualization layers.
imageRGB = image.visualize(**{'bands': ['B5', 'B4', 'B3'], 'max': 0.5})
ndwiRGB = ndwiMasked.visualize(
    **{'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}
)

Map.addLayer(imageRGB, {}, 'imageRGB')
Map.addLayer(ndwiRGB, {}, 'ndwiRGB')


# Display the map
Map

Map(center=[37.501, -122.1899], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

## Mosaicking

You can use masking and imageCollection.mosaic() (see the Mosaicking section for information on mosaicking) to achieve various cartographic effects. The mosaic() method renders layers in the output image according to their order in the input collection. The following example uses mosaic() to combine the masked NDWI and the false color composite and obtain a new visualization.

In this example, observe that a list of the two visualization images is provided to the ImageCollection constructor. The order of the list determines the order in which the images are rendered on the map. The result should look something like below.

In [8]:
# Mosaic the visualization layers and display (or export).
mosaic = ee.ImageCollection([imageRGB, ndwiRGB]).mosaic()
Map.addLayer(mosaic, {}, 'mosaic')

# Display the map
Map

Map(bottom=101876.0, center=[37.501, -122.1899], controls=(WidgetControl(options=['position', 'transparent_bg'…

## Clipping

The image.clip() method is useful for achieving cartographic effects. The following example clips the mosaic shown above to an arbitrary buffer zone around the city of San Francisco.

Note that the coordinates are provided to the Geometry constructor and the buffer length is specified as 20,000 meters. Learn more about geometries on the Geometries page. The result, shown with the map in the background, should look something like below.

In [9]:
# Create a circle by drawing a 20000 meter buffer around a point.
roi = ee.Geometry.Point([-122.4481, 37.7599]).buffer(20000)
clipped = mosaic.clip(roi)

# Display a clipped version of the mosaic.
Map.addLayer(clipped, {}, 'Clipped image')

# Display the map
Map

Map(bottom=102022.0, center=[37.34177550214861, -121.96060180664064], controls=(WidgetControl(options=['positi…

## Rendering categorical maps

Palettes are also useful for rendering discrete valued maps, for example a land cover map. In the case of multiple classes, use the palette to supply a different color for each class. (The image.remap() method may be useful in this context, to convert arbitrary labels to consecutive integers). The following example uses a palette to render land cover categories:

In [10]:
# Load 2012 MODIS land cover and select the IGBP classification.
cover = ee.Image('MODIS/051/MCD12Q1/2012_01_01').select('Land_Cover_Type_1')

# Define a palette for the 18 distinct land cover classes.
igbpPalette = [
    'aec3d4',  # water
    '152106',
    '225129',
    '369b47',
    '30eb5b',
    '387242',  # forest
    '6a2325',
    'c3aa69',
    'b76031',
    'd9903d',
    '91af40',  # shrub, grass
    '111149',  # wetlands
    'cdb33b',  # croplands
    'cc0013',  # urban
    '33280d',  # crop mosaic
    'd7cdcc',  # snow and ice
    'f7e084',  # barren
    '6f6f6f',  # tundra
]
# Specify the min and max labels and the color palette matching the labels.
Map.setCenter(-99.229, 40.413, 5)
Map.addLayer(
    cover, {'min': 0, 'max': 17, 'palette': igbpPalette}, 'IGBP classification'
)

Map

Map(bottom=202967.0, center=[40.413, -99.229], controls=(WidgetControl(options=['position', 'transparent_bg'],…

## Thumbnail images

Use the ee.Image.getThumbURL() method to generate a PNG or JPEG thumbnail image for an ee.Image object. Printing the outcome of an expression ending with a call to getThumbURL() results in a URL being printed to the console. Visiting the URL sets Earth Engine servers to work on generating the requested thumbnail on-the-fly. The image is displayed in the browser when processing completes. It can be downloaded by selecting appropriate options from the image’s right-click context menu.

The getThumbURL() method shares parameters with Map.addLayer(), described in the visualization parameters table above. Additionally, it takes optional dimension, region, and crs arguments that control the spatial extent, size, and display projection of the thumbnail.

A single-band image will default to grayscale unless a palette argument is supplied. A multi-band image will default to RGB visualization of the first three bands, unless a bands argument is supplied. If only two bands are provided, the first band will map to red, the second to blue, and the green channel will be zero filled.

The following are a series of examples demonstrating various combinations of getThumbURL() parameter arguments. Visit the URLs printed to the console when you run this script to view the thumbnails.

In [11]:
# Create a default map
Map = geemap.Map()

# Load an image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

# Define the visualization parameters.
vizParams = {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 0.5, 'gamma': [0.95, 1.1, 1]}

# Center the map and display the image.
Map.setCenter(-122.1899, 37.5010, 8)  # San Francisco Bay
Map.addLayer(image, vizParams, 'false color composite')

# Display the map
Map

Map(center=[37.501, -122.1899], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(ch…

In [12]:
geemap.get_image_thumbnail(image, "landsat.png", vizParams, dimensions=1000)